In [ ]:
import os
import requests
from requests.exceptions import HTTPError, ConnectionError, Timeout

# Load API key from an environment variable or a configuration file
api_key = 'insert the key'

def get_recipe_information(recipe_id, api_key, include_nutrition=True):
    try:
        include_nutrition_str = 'true' if include_nutrition else 'false'
        info_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?includeNutrition={include_nutrition_str}&apiKey={api_key}"
        
        response = requests.get(info_url)
        response.raise_for_status()

        return response.json()
    
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

    return None

def get_recipes_by_ingredients(api_key):
    try:
        ingredients = input("Enter the ingredients you have available (separated by commas): ")
        find_url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={ingredients}&number=5&apiKey={api_key}"
        
        response = requests.get(find_url)
        response.raise_for_status()

        return response.json()

    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

    return None

# Ask the user if they want to include nutrition data
include_nutrition = input("Would you like to include nutrition data? (yes/no): ").lower() == 'yes'

# Fetch recipes based on user input
recipes = get_recipes_by_ingredients(api_key)

# Check if recipes were found
if recipes:
    print("\nHere are some recipes based on the ingredients you have:")
    
    for recipe in recipes:
        recipe_info = get_recipe_information(recipe['id'], api_key, include_nutrition)
        
        if recipe_info:
            print(f"\nRecipe: {recipe_info['title']}")
            print(f"Link: {recipe_info.get('sourceUrl', 'No URL available')}")
            print(f"Missing Ingredients: {', '.join([ingredient['name'] for ingredient in recipe['missedIngredients']])}")

            if include_nutrition and 'nutrition' in recipe_info:
                print("Nutritional Information (per serving):")
                for nutrient in recipe_info['nutrition']['nutrients']:
                    print(f"- {nutrient['name']}: {nutrient['amount']}{nutrient['unit']}")
            else:
                print("Nutritional information is not included.")
else:
    print("No recipes found. Please try different ingredients.")
